In [ ]:
import PIL
import requests
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler

model_id = "timbrooks/instruct-pix2pix"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision="fp16", safety_checker=None)
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(device)
pipe.to(device)
pipe.enable_attention_slicing()

: 

In [ ]:
import traceback
def new_forward(self, *args, **kwargs):
    
    print ("\n************FORWARD HO**************\n")

    print(f'input tensor shape: {[arg.size() for arg in args]}')
    print(f'kwargs: {kwargs}')
    print(f'call stack: ')

    for line in traceback.format_stack():
        print(line.strip())

    return self.text_model(*args, **kwargs)

In [ ]:
image = PIL.Image.open("./david.jpeg")
image = PIL.ImageOps.exif_transpose(image)
image = image.convert("RGB")
image

In [ ]:

model = pipe.text_encoder.to(device)

forward_method_type = type(model.forward)

model.forward = forward_method_type(new_forward, model)

pipe.text_encoder = model

In [ ]:
seed = 10101
generator = torch.manual_seed(seed)
prompt = "make his face more smug"
smug = pipe(prompt, image=image, num_inference_steps=30, image_guidance_scale=1.05, generator=generator).images[0]
smug